# Connect four with Deep Q-Network vs Random agent from Tianshou

In this notebook, the created custom gym environment of connect four is played using a Deep Q-Network (DQN) agent which plays against a random agent.
We use the powerful [Tianshou library](https://github.com/thu-ml/tianshou) for this.
Using Tianshou in a multi-agent environment like ours meant we had to create yet another version of the Gym environment now based on an `AECEnv` environment.

The methodology used in this notebook is largely equal to those of the random agents in the `1-learning-connect-four-random-agents-tianshou.ipynb` notebook and the tutorial on [multi-agent tic-tac-toe learning from Tianshou](https://tianshou.readthedocs.io/en/master/tutorials/tictactoe.html).


<hr><hr>

## Table of Contents

- Contact information
- Checking requirements
  - Correct Anaconda environment
  - Correct module access
  - Correct CUDA access
- Training DQN vs Random agent on connect four Gym
  - Create the Gym environment
  - Building the environment
  - Implementing the DQN policy
  - Building agents
  - Function for letting agents learn
  - Function for watching learned agent
  - Doing the experiment
- Discussion

<hr><hr>

## Contact information

| Name             | Student ID | VUB mail                                                  | Personal mail                                               |
| ---------------- | ---------- | --------------------------------------------------------- | ----------------------------------------------------------- |
| Lennert Bontinck | 0568702    | [lennert.bontinck@vub.be](mailto:lennert.bontinck@vub.be) | [info@lennertbontinck.com](mailto:info@lennertbontinck.com) |



<hr><hr>

## Checking requirements

### Correct Anaconda environment

The `rl-project` anaconda environment should be active to ensure proper support. Installation instructions are available on [the GitHub repository of the RL course project and homeworks](https://github.com/pikawika/vub-rl).

In [ ]:
####################################################
# CHECKING FOR RIGHT ANACONDA ENVIRONMENT
####################################################

import os
from platform import python_version

print(f"Active environment: {os.environ['CONDA_DEFAULT_ENV']}")
print(f"Correct environment: {os.environ['CONDA_DEFAULT_ENV'] == 'rl-project'}")
print(f"\nPython version: {python_version()}")
print(f"Correct Python version: {python_version() == '3.8.10'}")

<hr>

### Correct module access

The following code block will load in all required modules and show if the versions match those that are recommended.

In [ ]:
####################################################
# LOADING MODULES
####################################################

# Allow reloading of libraries
import importlib

# Plotting
import matplotlib; print(f"Matplotlib version (3.5.1 recommended): {matplotlib.__version__}")
import matplotlib.pyplot as plt

# Argparser
import argparse

# More data types
import typing
import numpy as np

# Pygame
import pygame; print(f"Pygame version (2.1.2 recommended): {pygame.__version__}")

# Gym environment
import gym; print(f"Gym version (0.21.0 recommended): {gym.__version__}")

# Tianshou for RL algorithms
import tianshou as ts; print(f"Tianshou version (0.4.8 recommended): {ts.__version__}")

# Torch is a popular DL framework
import torch; print(f"Torch version (1.12.0 recommended): {torch.__version__}")

# PPrint is a pretty print for variables
from pprint import pprint

# Our custom connect four gym environment
import sys
sys.path.append('../')
import gym_connect4_pygame.envs.ConnectFourPygameEnvV2 as cfgym
importlib.invalidate_caches()
importlib.reload(cfgym)

# Time for allowing "freezes" in execution
import time;

# Allow for copying objects in a non reference manner
import copy

# Used for updating notebook display
from IPython.display import clear_output

In [ ]:
####################################################
# FUNCTION FOR LOADING IN TORCH DICTIONARIIES
####################################################

def load_torch_dict(filename):
    """
    Loads in torch dictionary using correct cuda settings for current device
    """   
    if torch.cuda.is_available():
        return torch.load(filename)
    else:
        return torch.load(filename, map_location=torch.device('cpu'))

<hr>

### Correct CUDA access

The installation instructions specify how to install PyTorch with CUDA 11.6.
The following code block tests if this was done successfully.

In [ ]:
####################################################
# CUDA VALIDATION
####################################################

# Check cuda available
print(f"CUDA is available: {torch.cuda.is_available()}")

# Show cuda devices
print(f"\nAmount of connected devices supporting CUDA: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    # Show current cuda device
    print(f"\nCurrent CUDA device: {torch.cuda.current_device()}")

    # Show cuda device name
    print(f"Cuda device 0 name: {torch.cuda.get_device_name(0)}")

<hr><hr>

## Training DQN vs Random agent on connect four Gym

Our connect four gym setup requires two agents, one for each player.
To reduce complexity, agents will always play as the same player, e.g. always as player 1.
It is important to note that connect four is a *solved game*.
According to [The Washington Post](https://www.washingtonpost.com/news/wonk/wp/2015/05/08/how-to-win-any-popular-game-according-to-data-scientists/):

> Connect Four is what mathematicians call a "solved game," meaning you can play it perfectly every time, no matter what your opponent does. You will need to get the first move, but as long as you do so, you can always win within 41 moves.

<hr>

### Create the Gym environment

Whilst our first connect four implementation (V1) was playable in a multi-agent manner through a manual game loop, as is done for random agents in the experimental notebook `2-testing-custom-gym-environment.ipynb`, it is hard to use most of the popular RL libraries for this environment.
That is because the Gym environment was originally made with single agent games in mind and there is no real standard on how to write multi-agent environments.
Thus, Tianshou, which offers some multi-agent support, didn't work well with this version of the Gym environment.
To tackle this, we created a V2, which is a rework of V1 to follow the standards of a *Petting Zoo* environment.
[Petting zoo](https://www.pettingzoo.ml/) is a library that offers many Gym environments extended to be multi-agent which uses wrapper classes and base classes so that each multi-agent environment follows the same guidelines.

These guidelines are exactly what V2 of our connect four implementation is adopted towards.
The environment is now a subclass of `AECEnv` rather than a `gym.Env`, which follows a similar approach but requires far more attributes and more complex observation and action spaces so that each agent has its own, even if they are all equal.

We note that we only found out about this library after running into trouble making our V1 work with libraries that support Gym training.
We could have searched for online implementations of e.g. Deep Q-Network (DQN) that allows training using the gym gaming loop used in the experimental notebook `2-testing-custom-gym-environment.ipynb`. 
Nonetheless, this would likely require a lot of manual work which is not ideal for the goal of this research, which focuses on reducing work for game developers.
Thus, the decision was made to create this V2 of the environment based on Petting Zoo environments.

As a final note, we noticed there was a Connect Four implementation of Petting Zoo.
This implementation differs from ours in several ways.
Their observation space consists of multiple variants of the board, namely one with only the opponent's coin and the agent's coins and it uses action masks to not allow placing coins in full columns.
We use a singular observation space of the complete board, which is the location of both agent's coins and the free spaces, corresponding to what a human would see.
Adding to this, we also don't programmatically disallow placing a coin in a full column but rather punish the user for trying that action and leave the board unchanged and let the agent play again.
There are more differences then the ones listed here and the visual game is also completely different.

In [ ]:
####################################################
# SETTING UP THE GYM ENVIRONMENT
####################################################

# Create an instance of the environment to be used
# V2 is used as this contains edits for Tianshou
# We use the Tianshou PettingZooEnv wrapper for multiagent support
env = ts.env.PettingZooEnv(cfgym.env())

# Get information about the environment
print(f"Observation space: {env.observation_space}")
print(f"\nAction space: {env.action_space}")

# Reset the environment to start from a clean state, returns the initial observation
observation = env.reset()

print("\n Initial player id:")
print(observation["agent_id"])

print("\n Initial observation:")
print(observation["obs"])

print("\n Initial mask:")
print(observation["mask"])

# Clean unused variables
del observation
del env

<hr>

### Building the environment

We build a simple function to get the environment the agent will play in

In [ ]:
####################################################
# CONNECT FOUR V2 ENVIRONMENT
####################################################

def get_env():
    """
    Returns the connect four gym environment V2 altered for Tianshou and Petting Zoo compatibility.
    Already wrapped with a ts.env.PettingZooEnv wrapper.
    """
    return ts.env.PettingZooEnv(cfgym.env())

<hr>

### Implementing the DQN policy

The DQN policy for the agent is configured and set up below.

In [ ]:
####################################################
# DQN POLICY
####################################################

def cf_dqn_policy(state_shape: tuple,
                  action_shape: tuple,
                  optim: typing.Optional[torch.optim.Optimizer] = None,
                  learning_rate: float =  0.0001,
                  gamma: float = 0.7, # Smaller gamma favours "faster" win
                  n_step: int = 3, # Number of steps to look ahead
                  target_update_freq: int = 320,
                  hidden_sizes: list = [128, 128, 128, 128]):
    # Use cuda device if possible
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Network to be used for DQN
    ## Set to use the CUDA device if possible
    net = ts.utils.net.common.Net(state_shape= state_shape,
                                  action_shape= action_shape,
                                  hidden_sizes= hidden_sizes,
                                  device= device).to(device)
    
    # Default optimizer is an adam optimizer with the argparser learning rate
    if optim is None:
        optim = torch.optim.Adam(net.parameters(), lr= learning_rate)
        
    # Our agent DQN policy
    return ts.policy.DQNPolicy(model= net,
                               optim= optim,
                               discount_factor= gamma,
                               estimation_step= n_step,
                               target_update_freq= target_update_freq)

<hr>

### Building agents

We can now adopt the `get_agents` function from Tianshou to build our DQN agents for the game.

NOTE: agent 2 is random for now whilst agent 1 will use DQN.

In [ ]:
####################################################
# AGENT CREATION
####################################################

def get_agents(agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
               agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
               optim: typing.Optional[torch.optim.Optimizer] = None,
               resume_path_player_1: str = '', # Path to file to resume agent training from
               ) -> typing.Tuple[ts.policy.BasePolicy, torch.optim.Optimizer, list]:
    """
    Gets a multi agent policy manager, optimizer and player ids for the connect four V2 gym environment.
    Per default this returns 
        - Multi agent manager for agent 1 using DQN and agent 2 using random policy
        - Adam optimizer
        - ['player_1', 'player_2'] from the connect four environment
    """
    
    # Get the environment to play in (Connect four gym V2)
    env = get_env()
    
    # Get the observation space from the environment, depending on typo of space (ternary operator)
    observation_space = env.observation_space['observation'] if isinstance(env.observation_space, gym.spaces.Dict) else env.observation_space
    
    # Set the arguments
    state_shape = observation_space.shape or observation_space.n
    action_shape = env.action_space.shape or env.action_space.n
    
    # Configure agent player 1 to be a DQN if no policy is passed.
    if agent_player1 is None:
        # Our agent1 uses a DQN policy
        agent_player1 = cf_dqn_policy(state_shape= state_shape,
                                      action_shape= action_shape,
                                      optim= optim)
        
        # If we resume our agent we need to load the previous config
        if resume_path_player_1:
            agent_player1.load_state_dict(load_torch_dict(resume_path_player_1))

    # Configure agent player 2 to be a DQN if no policy is passed
    #   or a copy of player 1 with the specified path.
    if agent_player2 is None:
        # Our agent2 uses a random policy per default for now
        agent_player2 = ts.policy.RandomPolicy()

    # Player 1 agent is DQN, other is random
    agents = [agent_player1, agent_player2]
        
    # Our policy depends on the order of the agents
    policy = ts.policy.MultiAgentPolicyManager(agents, env)
    
    # Return our policy, optimizer and the available agents in the environment
    # Per default: 
    #   - Multi agent manager for agent 1 using DQN and agent 2 using random policy
    #   - Adam optimizer
    #   - ['player_1', 'player_2'] from the connect four environment
    
    return policy, optim, env.agents

<hr>

### Function for letting agents learn

We can now adopt the `train_agent` function from Tianshou to allow our DQN and random agent for the game to learn.

In [ ]:
####################################################
# AGENT TRAINING
####################################################

def train_agent(filename: str = "dqn_vs_random",
                agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
                agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
                optim: typing.Optional[torch.optim.Optimizer] = None,
                training_env_num: int = 10,
                testing_env_num: int = 10,
                buffer_size: int = 2^14, # 16 384
                batch_size: int = 64,
                epochs: int = 50,
                step_per_epoch: int = 1000,
                step_per_collect: int = 10,
                update_per_step: float = 0.1,
                testing_eps: float = 0.05,
                training_eps: float = 0.1,
                ) -> typing.Tuple[dict, ts.policy.BasePolicy]:
    """
    Trains two agents in the connect four V2 environment and saves their best model and logs.
    Returns:
        - result from offpolicy_trainer
        - final version of agent 1
        - final version of agent 2
    """

    # ======== environment setup =========
    train_envs = ts.env.DummyVectorEnv([get_env for _ in range(training_env_num)])
    test_envs = ts.env.DummyVectorEnv([get_env for _ in range(testing_env_num)])
    
    # set the seed for reproducibility
    np.random.seed(1998)
    torch.manual_seed(1998)
    train_envs.seed(1998)
    test_envs.seed(1998)

    # ======== agent setup =========
    # Gets our agents from the previously made function
    # Per default: 
    #   - Multi agent manager for agent 1 using DQN and agent 2 using random policy
    #   - Adam optimizer
    #   - ['player_1', 'player_2'] from the connect four environment
    policy, optim, agents = get_agents(agent_player1=agent_player1,
                                       agent_player2=agent_player2,
                                       optim=optim)

    # ======== collector setup =========
    # Make a collector for the training environments
    train_collector = ts.data.Collector(policy= policy,
                                        env= train_envs,
                                        buffer= ts.data.VectorReplayBuffer(buffer_size, len(train_envs)),
                                        exploration_noise= True)
    
    # Make a collector for the testing environments
    test_collector = ts.data.Collector(policy= policy,
                                       env= test_envs,
                                       exploration_noise= True)
    
    # Uncomment below if you want to set epsilon in epsilon policy
    # policy.set_eps(1)
    
    # Collect data fot the training evnironments
    train_collector.collect(n_step= batch_size * training_env_num)
    
    # ======== ensure folders exist =========
    if not os.path.exists(os.path.join('./logs', 'paper_notebooks', '2', filename)):
        os.makedirs(os.path.join('./logs', 'paper_notebooks', '2', filename))
    if not os.path.exists(os.path.join('./saved_variables', 'paper_notebooks', '2', filename)):
        os.makedirs(os.path.join('./saved_variables', 'paper_notebooks', '2', filename))

    # ======== tensorboard logging setup =========
    # Allows to save the training progress to tensorboard compatable logs
    log_path = os.path.join('./logs', 'paper_notebooks', '2', filename)
    writer = torch.utils.tensorboard.SummaryWriter(log_path)
    logger = ts.utils.TensorboardLogger(writer)

    # ======== callback functions used during training =========
    # We want to save our best policy
    def save_best_fn(policy):
        """
        Callback to save the best model
        """
        # Save agent 1
        model_save_path = os.path.join('./saved_variables', 'paper_notebooks', '2', filename, 'best_policy_agent1.pth')
        torch.save(policy.policies[agents[0]].state_dict(), model_save_path)
        
        # Save agent 2
        model_save_path = os.path.join('./saved_variables', 'paper_notebooks', '2', filename, 'best_policy_agent2.pth')
        torch.save(policy.policies[agents[1]].state_dict(), model_save_path)
        
        # Save agent2

    def stop_fn(mean_rewards):
        """
        Callback to stop training when we've reached the win rate
        """
        return mean_rewards >= 7 # (win = 10, 70% win without invalid moves = mean of 7)

    def train_fn(epoch, env_step):
        """
        Callback for training
        """        
        # Agent 1
        policy.policies[agents[0]].set_eps(training_eps)

    def test_fn(epoch, env_step):
        """
        Callback for testing
        """        
        # Agent 1
        policy.policies[agents[0]].set_eps(testing_eps)

    def reward_metric(rews):
        """
        Callback for reward collection
        """
        # We are only interested in optimizing agent 1 and thus only agent 1 score
        return rews[:, 0]

    # trainer
    result = ts.trainer.offpolicy_trainer(policy= policy,
                                          train_collector= train_collector,
                                          test_collector= test_collector,
                                          max_epoch= epochs,
                                          step_per_epoch= step_per_epoch,
                                          step_per_collect= step_per_collect,
                                          episode_per_test= testing_env_num,
                                          batch_size= batch_size,
                                          train_fn= train_fn,
                                          test_fn= test_fn,
                                          # Stop function to stop before specified amount of epochs
                                          #stop_fn= stop_fn
                                          save_best_fn= save_best_fn,
                                          update_per_step= update_per_step,
                                          logger= logger,
                                          test_in_train= False,
                                          reward_metric= reward_metric)

    return result, policy.policies[agents[0]], policy.policies[agents[1]]

<hr>

### Function for watching learned agent

When an agent has learned, we can watch the learned policy in action.

In [ ]:
####################################################
# WATCHING THE LEARNED POLICY IN ACTION
####################################################

def watch(numer_of_games:int = 3,
          agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
          agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
          test_epsilon: float = 0.05, # For the watching we act completely greedy but low random for not getting stuck on invalid move
          render_speed: float = 0.15, # Amount of seconds to update frame/ do a step
          ) -> None:
    
    # Get the connect four V2 environment (must be a list)
    env= ts.env.DummyVectorEnv([get_env])
    
    # Get the agents from the trained agents
    policy, optim, agents = get_agents(agent_player1= agent_player1,
                                       agent_player2= agent_player2)
    
    # Evaluate the policy
    policy.eval()
    
    # Set the testing policy epsilon for our first agent
    policy.policies[agents[0]].set_eps(test_epsilon)
    
    # Collect the test data
    collector = ts.data.Collector(policy= policy,
                                  env= env,
                                  exploration_noise= True)
    
    # Render games in human mode to see how it plays
    result = collector.collect(n_episode= numer_of_games, render= render_speed)
    
    # Close the environment aftering collecting the results
    # This closes the pygame window after completion
    env.close()
    
    # Get the rewards and length from the test trials
    rewards, length = result["rews"], result["lens"]
    
    # Print the final reward for the first agent
    print(f"Average steps of game:  {length.mean()}")
    print(f"Final mean reward agent 1: {rewards[:, 0].mean()}, std: {rewards[:, 0].std()}")
    print(f"Final mean reward agent 2: {rewards[:, 1].mean()}, std: {rewards[:, 1].std()}")

<hr>

### Doing the experiment

We now do the experiment with using our previously created functions.

In [ ]:
####################################################
# EXPERIMENT: TRAINING AGENTS
####################################################

# Train the agent
off_policy_traininer_results, final_agent_player1, final_agent_player2 = train_agent(epochs= 50)

In [ ]:
####################################################
# EXPERIMENT: VIEWING THE LEARNED POLICY
####################################################

# Get the environment settings
env = get_env()
observation_space = env.observation_space['observation'] if isinstance(env.observation_space, gym.spaces.Dict) else env.observation_space
state_shape = observation_space.shape or observation_space.n
action_shape = env.action_space.shape or env.action_space.n

# Configure the best agent
best_agent1 = cf_dqn_policy(state_shape= state_shape,
                            action_shape= action_shape)
best_agent1.load_state_dict(load_torch_dict("./saved_variables/paper_notebooks/2/dqn_vs_random/best_policy_agent1.pth"))

# Watch the best agetn at work
watch(numer_of_games= 3,
      agent_player1= best_agent1)



<hr><hr>

## Discussion

The DQN agent has learned to stack coins in the hope that the other agent won't stack in that column and thus he will get a quick win.
This is a decent strategy against a random opponent but not that good against an actual human.
This also results in the agent getting stuck in a situation where it keeps stacking coins onto a full row, collecting many negative rewards.
This is likely to be resolved if more training is done but to make things more challenging for the smart DQN agent we will let him play against another DQN agent in the next notebook.

In [ ]:
####################################################
# CLEAN VARIABLES
####################################################

del action_shape
del best_agent1
del env
del final_agent_player1
del final_agent_player2
del observation_space
del off_policy_traininer_results
del state_shape
